In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import gc

def flatten_conditional_feature(f):
    return [[x.split(':')[0]] + x.split(':')[1].split(',') for x in f.split(' ')]

data_path = "./data/traffic/"

for date in tqdm(range(1,32)):
    with open(data_path + '201907%02d.txt'%date) as file:
        line = file.readline().replace('\n','')
        feature_dict = {"date": date - 31,
                  "link": [],
                  "label": [],
                  "current_time": [],
                  "predict_time": []}

        while line:
            index = line.split(';')[0].split(' ')
            recent_feature = flatten_conditional_feature(line.split(';')[1])
            history_features = []
            for conditonal_feature in line.split(';')[2:]:
                history_features.append(flatten_conditional_feature(conditonal_feature))

            #generate features for DF    
            feature_dict['link'].append(index[0])
            feature_dict['label'].append(index[1])
            feature_dict['current_time'].append(index[2])
            feature_dict['predict_time'].append(index[3])

            for t in range(0, 5):
                for i,name in enumerate(['feature_time',"feature_speed",'feature_eta','feature_status','feature_carnums']):
                    key = 'current_' + name + '_%d'%(t+1)
                    feature_dict.setdefault(key, [])
                    feature_dict[key].append(recent_feature[t][i])
            for w in range(4):
                for t in range(0, 5):
                    for i,name in enumerate(['feature_time',"feature_speed",'feature_eta','feature_status','feature_carnums']):
                        key = 'history%d_'%(w + 1) + name + '_%d'%(t+1)
                        feature_dict.setdefault(key, [])
                        feature_dict[key].append(history_features[w][t][i])
            line = file.readline().replace('\n','')
        pd.DataFrame(feature_dict).to_csv(data_path + 'train_table_%02d.gz'%date, index=False, compression='gzip')
        feature_dict = None
        gc.collect()

100%|██████████| 10/10 [40:27<00:00, 242.80s/it]
